In [1]:
from transformers import T5Tokenizer,T5ForConditionalGeneration
import torch

tokenizer = T5Tokenizer.from_pretrained("t5-small")
rl_model=T5ForConditionalGeneration.from_pretrained('./ppo_MB_sd_di20e_wtest/model')
rl_model.eval()

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

In [2]:

def optimize_one_text(u_model,one_text):
    inputs = tokenizer(one_text, return_tensors="pt")
    with torch.no_grad():
        outputs = u_model.generate(
        input_ids=inputs.input_ids,
        attention_mask=inputs.attention_mask,
        max_length=50,
        early_stopping=True)
    result = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return result

In [3]:
import pandas as pd
compare_promts = pd.read_parquet('blip2_ft_sd/result/other_generative_model_test.parquet')

In [4]:
from tqdm import tqdm
rl_generated_caption=[]
generated_captions=compare_promts['generated_caption'].tolist()
len_generation=len(generated_captions)
for i in tqdm(range(len_generation)):
    rl_generated_caption.append(optimize_one_text(rl_model,generated_captions[i]))

  0%|          | 0/300 [00:00<?, ?it/s]/home/fenghe/anaconda3/envs/RL4LM/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:649: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
100%|██████████| 300/300 [01:31<00:00,  3.26it/s]


In [5]:
compare_promts['rl_generation']=rl_generated_caption

compare_promts.to_parquet(f'blip2_ft_sd/result/rl_other_generative_model_test.parquet')
compare_promts

,generated_caption,reference_prompt,img_path,rl_generation
0,a beautiful fantasy digital painting of a Nors...,a beautiful Norse Warrior based on Hades with ...,./imgs/image_2700.png,"Greg Rutkowski and Raymond Swanland, a fantasy..."
1,"a blueprints of new york city, concept art, co...","blueprints for New York City, concept art, blu...",./imgs/image_4113.png,"a blueprints of new york city, concept art, by..."
2,"a spider with eyes of blood, concept art, high...","d d monster, huge spider monster covered in ey...",./imgs/image_3278.png,"a spider with eyes of blood, highly detailed, ..."
3,"concept art of medieval knight armor, 4 k, hig...","different views of medieval knights, beautiful...",./imgs/image_3909.png,"Concept art of medieval knight armor, 4 k, hig..."
4,lofi underwater bioshock steampunk portrait of...,"lofi underwater steampunk bioshock bikini, oct...",./imgs/image_2164.png,Portrait lofi underwater bioshock steampunk po...
...,...,...,...,...
295,"a beautiful painting of a medieval knight, by ...",a robot wearing medieval armor in the style of...,./imgs/image_4086.png,a painting of a medieval knight who looks like...
296,steampunk portrait of a man with a top hat and...,"Lofi Steampunk portrait, Pixar style, by Trist...",./imgs/image_3450.png,a steampunk portrait of a top sleazy man weari...
297,a beautiful painting of a dinosaur in a candyl...,"A dinosaur in a candy forest, digital art, tre...",./imgs/image_2398.png,a beautiful painting of a dinosaur in a candyl...
298,"a beautiful painting of a cute cat smiling, bl...",cute blue striped cat of cheshire from alice i...,./imgs/image_1770.png,A cute blue cat in the background smiling as a...
